# Pesquisa CBGG 2025

## Import de bibliotecas

In [1]:
from pysus import SIH
from pysus.ftp import FTPSingleton
import pandas as pd
import source.graphfile as gf

## Download e carregamento dos Dados

In [2]:
FTPSingleton.timeout = 180
print("Carregando arquivos do SIH...")
sih = SIH().load()
print("Arquivos do SIH carregados!")

files = sih.get_files("RD", uf="CE",
year=[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023],
month=[1,2,3,4,5,6,7,8,9,10,11,12])

print("Baixando arquivos do SIH...")
arquivos = sih.download(files,local_dir="./dados/DBC")
print("Arquivos do SIH baixados!")
print("Gerando dataframe dos dados do SIH...")
dataframes = []
for arquivo in arquivos:
    df = arquivo.to_dataframe()
    dataframes.append(df)
    print(f"df {arquivo} gerado!")
df_sih = pd.concat(dataframes, ignore_index=True)
df_sih.to_csv("dados/dados_sih.csv")
print("Datafrane do SIH gerado!")


Carregando arquivos do SIH...
Arquivos do SIH carregados!
Baixando arquivos do SIH...


3259775it [00:00, 162889873.61it/s]    

Arquivos do SIH baixados!
Gerando dataframe dos dados do SIH...


df dados/DBC/RDCE1401.parquet gerado!
df dados/DBC/RDCE1402.parquet gerado!
df dados/DBC/RDCE1403.parquet gerado!
df dados/DBC/RDCE1404.parquet gerado!
df dados/DBC/RDCE1405.parquet gerado!
df dados/DBC/RDCE1406.parquet gerado!
df dados/DBC/RDCE1407.parquet gerado!
df dados/DBC/RDCE1408.parquet gerado!
df dados/DBC/RDCE1409.parquet gerado!
df dados/DBC/RDCE1410.parquet gerado!
df dados/DBC/RDCE1411.parquet gerado!
df dados/DBC/RDCE1412.parquet gerado!
df dados/DBC/RDCE1501.parquet gerado!
df dados/DBC/RDCE1502.parquet gerado!
df dados/DBC/RDCE1503.parquet gerado!
df dados/DBC/RDCE1504.parquet gerado!
df dados/DBC/RDCE1505.parquet gerado!
df dados/DBC/RDCE1506.parquet gerado!
df dados/DBC/RDCE1507.parquet gerado!
df dados/DBC/RDCE1508.parquet gerado!
df dados/DBC/RDCE1509.parquet gerado!
df dados/DBC/RDCE1510.parquet gerado!
df dados/DBC/RDCE1511.parquet gerado!
df dados/DBC/RDCE1512.parquet gerado!
df dados/DBC/RDCE1601.parquet gerado!
df dados/DBC/RDCE1602.parquet gerado!
df dados/DBC

In [3]:
cnes_mapping = pd.read_csv("dados/CNESBR.csv")
munic_mapping = pd.read_csv("dados/MUNICSBR.csv")
df = pd.read_csv("dados/dados_sih.csv")

# Garantir que os códigos estão no mesmo formato (string)
df["CNES"] = df["CNES"].astype(str)
df["MUNIC_RES"] = df["MUNIC_RES"].astype(str)
cnes_mapping["CNES"] = cnes_mapping["CNES"].astype(str)
munic_mapping["COD"] = munic_mapping["COD"].astype(str)

# Substituir os códigos de CNES pelos nomes
df = df.merge(cnes_mapping.rename(columns={"NOMEFANT": "HOSPITAL"}), how="left", on="CNES")

# Substituir os códigos de MUNIC_RES pelos nomes
df = df.merge(munic_mapping.rename(columns={"MUNIC": "MUNICIPIO", "COD": "MUNIC_RES"}), how="left", on="MUNIC_RES")

# Exibir algumas linhas para verificar
print(df.head())

/var/folders/rl/0xvl00cs0fsfh5_g7ggqx4qr0000gn/T/ipykernel_8324/979251331.py:3: DtypeWarning: Columns (42,78,79,88,89,96,97,98,99,100,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("dados/dados_sih.csv")


   Unnamed: 0   UF_ZI  ANO_CMPT  MES_CMPT  ESPEC      CGC_HOSP          N_AIH  \
0           0  230000      2014         1      7  5.268526e+12  2314100004788   
1           1  230000      2014         1      7  5.268526e+12  2314100004854   
2           2  230000      2014         1      7  5.268526e+12  2314100004865   
3           3  230000      2014         1      7  5.268526e+12  2314100004876   
4           4  230000      2014         1      7  5.268526e+12  2314100004898   

   IDENT       CEP MUNIC_RES  ...  TPDISEC2  TPDISEC3  TPDISEC4  TPDISEC5  \
0      1  62900000    231180  ...       NaN       NaN       NaN       NaN   
1      1  62755000    230945  ...       NaN       NaN       NaN       NaN   
2      1  60870576    230440  ...       NaN       NaN       NaN       NaN   
3      1  60851290    230440  ...       NaN       NaN       NaN       NaN   
4      1  60843250    230440  ...       NaN       NaN       NaN       NaN   

   TPDISEC6  TPDISEC7  TPDISEC8  TPDISEC9  \
0    

## Filtro dos Dados

In [4]:
def calcular_idade_convertida(row):
    if row['COD_IDADE'] == 5:  # Idade maior ou igual a 100 anos
        return 100 + row['IDADE']
    elif row['COD_IDADE'] == 2:  # Idade em dias
        return row['IDADE'] // 365  # Aproximar para anos
    elif row['COD_IDADE'] == 3:  # Idade em meses
        return row['IDADE'] // 12  # Aproximar para anos
    elif row['COD_IDADE'] == 4:  # Idade em anos
        return row['IDADE']
    else:
        return None  # Caso COD_IDADE seja inválido

# Criar a nova coluna 'idade_convertida'
df['idade_convertida'] = df.apply(calcular_idade_convertida, axis=1)

In [5]:
df["IDADE"] = pd.to_numeric(df["IDADE"], errors="coerce")
df_s72 = df[
    df["DIAG_PRINC"].isin(["S720", "S721", "S722"]) & 
    (df["idade_convertida"] >= 60)
]
df_s72 = df_s72.dropna(subset=["HOSPITAL", "MUNICIPIO"])

# Carregar o arquivo TB_SIGTAW.csv que contém os códigos e nomes dos procedimentos
tb_sig = pd.read_csv("dados/TB_SIGTAW.csv")

# Renomear as colunas de TB_SIGTAW.csv para facilitar o merge
tb_sig.rename(columns={"IP_COD": "PROC_REA", "IP_DSCR": "PROCEDIMENTO"}, inplace=True)

# Garantir que os tipos de dados sejam consistentes para o merge (caso necessário)
df_s72["PROC_REA"] = df_s72["PROC_REA"].astype(str)
tb_sig["PROC_REA"] = tb_sig["PROC_REA"].astype(str)

# Realizar o merge para adicionar a coluna 'PROCEDIMENTO'
df_s72 = df_s72.merge(tb_sig, on="PROC_REA", how="left")
print("AIHs total do CE: ",len(df_s72))

AIHs total do CE:  12589


In [6]:
df["IDADE"] = pd.to_numeric(df["IDADE"], errors="coerce")
# TODO: Levantar a diferença dos dados de Municipios Fora e Dentro do Estado do Ceará
df_s72_ce = df[
    df["DIAG_PRINC"].isin(["S720", "S721", "S722"]) & 
    df["MUNICIPIO"].str.endswith(", CE") & 
    (df["idade_convertida"] >= 60)
]
df_s72_ce = df_s72_ce.dropna(subset=["HOSPITAL", "MUNICIPIO"])

# Carregar o arquivo TB_SIGTAW.csv que contém os códigos e nomes dos procedimentos
tb_sig = pd.read_csv("dados/TB_SIGTAW.csv")

# Renomear as colunas de TB_SIGTAW.csv para facilitar o merge
tb_sig.rename(columns={"IP_COD": "PROC_REA", "IP_DSCR": "PROCEDIMENTO"}, inplace=True)

# Garantir que os tipos de dados sejam consistentes para o merge (caso necessário)
df_s72_ce["PROC_REA"] = df_s72_ce["PROC_REA"].astype(str)
tb_sig["PROC_REA"] = tb_sig["PROC_REA"].astype(str)

# Realizar o merge para adicionar a coluna 'PROCEDIMENTO'
df_s72_ce = df_s72_ce.merge(tb_sig, on="PROC_REA", how="left")
print("AIHs apenas de municipios do CE: ", len(df_s72_ce))

AIHs apenas de municipios do CE:  12527


## Plotar Grafo

In [7]:
# Carregar os dados geográficos
geo_data = pd.read_csv("./dados/geograf.csv")
grafoS72 = gf.criar_grafo(geo_data, df_s72)
gf.plotar_grafo_folium(grafoS72,output_path='./output/grafoS72.html')

Erro: Coordenadas não encontradas para o município LIMEIRA, SP
Erro: Coordenadas não encontradas para o município RIO DE JANEIRO, RJ
Erro: Coordenadas não encontradas para o município ARARIPINA, PE
Erro: Coordenadas não encontradas para o município RIO DE JANEIRO, RJ
Erro: Coordenadas não encontradas para o município IPUBI, PE
Erro: Coordenadas não encontradas para o município RIBEIRAO PIRES, SP
Erro: Coordenadas não encontradas para o município JOAO PESSOA, PB
Erro: Coordenadas não encontradas para o município SAO GONCALO DO AMARANTE, RN
Erro: Coordenadas não encontradas para o município RIBEIRAO PIRES, SP
Erro: Coordenadas não encontradas para o município RECIFE, PE
Erro: Coordenadas não encontradas para o município NATAL, RN
Erro: Coordenadas não encontradas para o município ARAIOSES, MA
Erro: Coordenadas não encontradas para o município JUAZEIRO, BA
Erro: Coordenadas não encontradas para o município JARDIM, MS
Erro: Coordenadas não encontradas para o município MATRIZ DE CAMARAGIBE,

## Gerar caracterização dos dados

### Tabela de caracterização

In [8]:
# Raça (ordenado crescentemente pela categoria)
raca = (df_s72["RACA_COR"].value_counts(normalize=True) * 100).round(1).reset_index()
raca.columns = ["Raça", "Percentual (%)"]
raca = raca.sort_values(by="Raça")  # Ordenar pela coluna "Raça"

# Sexo (ordenado crescentemente por categoria)
sexo = (df_s72["SEXO"].value_counts(normalize=True) * 100).round(1).reset_index()
sexo.columns = ["Sexo", "Percentual (%)"]
sexo = sexo.sort_values(by="Sexo")  # Ordenar pela coluna "Sexo"

# Estatísticas de Idade (não precisa de ordenação, apenas exibição)
idade_stats = pd.DataFrame({
    "Estatística": ["Mínima", "Máxima", "Média", "Mediana", "Moda"],
    "Idade": [
        int(df_s72["idade_convertida"].min()),
        int(df_s72["idade_convertida"].max()),
        round(df_s72["idade_convertida"].mean(), 1),
        int(df_s72["idade_convertida"].median()),
        int(df_s72["idade_convertida"].mode()[0])
    ]
})

# Faixa Etária (<60, 60-69, 70-79, 80-89, 90-99, 100+)
bins = [0, 59, 69, 79, 89, 99, float("inf")]
labels = ["<60", "60-69", "70-79", "80-89", "90-99", "100+"]
df_s72["FAIXA_ETÁRIA"] = pd.cut(df_s72["idade_convertida"], bins=bins, labels=labels, ordered=True)
faixa_etaria = (df_s72["FAIXA_ETÁRIA"].value_counts(normalize=True) * 100).round(1).reset_index()
faixa_etaria.columns = ["Faixa Etária", "Percentual (%)"]
faixa_etaria = faixa_etaria.sort_values(by="Faixa Etária")  # Ordenar pela coluna "Faixa Etária"

# Exibir as tabelas organizadas
print("Tabela de Raça:")
print(raca)

print("\nTabela de Sexo:")
print(sexo)

print("\nEstatísticas de Idade:")
print(idade_stats)

print("\nTabela de Faixa Etária:")
print(faixa_etaria)

Tabela de Raça:
   Raça  Percentual (%)
2     1             6.9
4     2             1.0
0     3            64.2
3     4             4.4
5     5             0.0
1    99            23.5

Tabela de Sexo:
   Sexo  Percentual (%)
1     1            26.4
0     3            73.6

Estatísticas de Idade:
  Estatística  Idade
0      Mínima   60.0
1      Máxima  113.0
2       Média   79.2
3     Mediana   80.0
4        Moda   82.0

Tabela de Faixa Etária:
  Faixa Etária  Percentual (%)
5          <60             0.0
2        60-69            16.4
1        70-79            32.7
0        80-89            38.6
3        90-99            11.8
4         100+             0.5


### Frequencia de procedimentos

In [14]:
frequencia_procedimento = df_s72["PROCEDIMENTO"].value_counts()
percentual_procedimento = df_s72["PROCEDIMENTO"].value_counts(normalize=True) * 100

# Criar o DataFrame com as frequências e percentuais
tabela_frequencias_procedimentos = pd.DataFrame({
    'Frequência': frequencia_procedimento,
    'Percentual (%)': percentual_procedimento
}).reset_index()

# Renomear a coluna para "PROCEDIMENTO"
tabela_frequencias_procedimentos.rename(columns={'index': 'PROCEDIMENTO'}, inplace=True)

# Adicionar os códigos dos procedimentos (PROC_REA)
tabela_frequencias_procedimentos = tabela_frequencias_procedimentos.merge(
    tb_sig[['PROC_REA', 'PROCEDIMENTO']],
    on='PROCEDIMENTO',
    how='left'
)

# Reordenar colunas para melhor visualização
tabela_frequencias_procedimentos = tabela_frequencias_procedimentos[['PROC_REA', 'PROCEDIMENTO', 'Frequência', 'Percentual (%)']]

tabela_frequencias_procedimentos.to_excel("output/freq_procedimentos.xlsx")
print(tabela_frequencias_procedimentos)

     PROC_REA                                       PROCEDIMENTO  Frequência  \
0   408050632  TRATAMENTO CIRURGICO DE FRATURA TRANSTROCANTER...        5084   
1   408040050                    ARTROPLASTIA PARCIAL DE QUADRIL        1521   
2   408050616   TRATAMENTO CIRURGICO DE FRATURA SUBTROCANTERIANA        1398   
3   408040092  ARTROPLASTIA TOTAL PRIMARIA DO QUADRIL NAO CIM...         950   
4   408050489  TRATAMENTO CIRURGICO DE FRATURA / LESAO FISARI...         789   
5   415010012                  TRATAMENTO C/ CIRURGIAS MULTIPLAS         734   
6   408040084   ARTROPLASTIA TOTAL PRIMARIA DO QUADRIL CIMENTADA         636   
7   308010019  TRATAMENTO CLINICO/CONSERVADOR DE TRAUMATISMOS...         504   
8   415030013           TRATAMENTO CIRURGICO EM POLITRAUMATIZADO         290   
9   303090138  TRATAMENTO CONSERVADOR DE FRATURA / LESAO LIGA...         272   
10  301060070  DIAGNOSTICO E/OU ATENDIMENTO DE URGENCIA EM CL...         144   
11  301060088  DIAGNOSTICO E/OU ATENDIME

### Frequencia de idade

In [10]:
frequencia_idade = df_s72["idade_convertida"].value_counts()
percentual_idade = df_s72["idade_convertida"].value_counts(normalize=True) * 100

tabela_frequencias_idades = pd.DataFrame({
    'Frequência': frequencia_idade,
    'Percentual (%)': percentual_idade
})

tabela_frequencias_idades.to_excel("output/freq_idades.xlsx")
print(tabela_frequencias_idades)

                  Frequência  Percentual (%)
idade_convertida                            
82                       569        4.519819
81                       548        4.353007
83                       534        4.241798
79                       520        4.130590
85                       513        4.074986
80                       513        4.074986
84                       513        4.074986
78                       488        3.876400
77                       468        3.717531
86                       463        3.677814
87                       459        3.646040
76                       443        3.518945
75                       429        3.407737
88                       421        3.344189
73                       403        3.201207
74                       402        3.193264
72                       349        2.772261
89                       330        2.621336
71                       315        2.502184
70                       301        2.390976
91        